In [1]:
import numpy as np

In [7]:
def play_game(p1, p2, env, draw=False):
    current_player = None
    
    # loops until the game is over
    while not env.game_over():
        # alternate between players (p1 always first)
        if current_player == p1:
            current_player = p2
        else:
            current_player = p1
        
        # make a move
        current_player.take_action(env)
        
        # draw if necessary
        if draw:
            if draw == 1 and current_player == p1:
                env.draw_board()
            if draw == 2 and current_player == p2:
                env.draw_board()
        
        # update histories
        state = env.get_state()
        p1.update_state_history(state)
        p2.update_state_history(state)
    
    if draw:
        env.draw_board()
    
    # value function update
    p1.update(env)
    p2.update(env)


In [3]:
LENGTH = 3

class Environment:
    def __init__(self):
        self.board = np.zeros((LENGTH, LENGTH))
        self.x = -1 # x on the board, player 1
        self.o = 1 # o on the board, player 2
        self.winner = None
        self.ended = False
        self.num_states = 3 ** (LENGTH*LENGTH)
    
    def is_empty(self, i, j):
        return self.board[i,j] == 0
        
    def reward(self, symbol):
        # no reward until game is over
        if not self.game_over():
            return 0
        
        # game is over, symbol is either self.x or self.o
        if self.winner == symbol:
            return 1
        else:
            return 0
        
    def get_state(self):
        # retruens the current state, represented as an int
        # S = set of all possibles states
        # |S| = 3 ^ (BOARD SIZE) [each cell has 3 possible values]
        # some satte are not possible
        # trinanry to decimal implementation
        k = 0
        h = 0
        for i in xrange(LENGTH):
            for j in xrange(LENGTH):
                if self.board[i,j] == 0:
                    v = 0
                elif self.board[i,j] == self.x:
                    v = 1
                elif self.board[i,j] == self.o:
                    v = 2
                h += (3**k) * v
                k += 1
        return h
    
    def game_over(self, force_recalculate=False):
        if not force_recalculate and self.ended:
            return self.ended
        
        # check rows
        for i in xrange(LENGTH):
            for play in (self.x, self.o):
                if self.board[i].sum() == player*LENGTH:
                    self.winner = player
                    self.ended = True
                    return True
        # check columns
        for j in xrange(LENGTH):
            for play in (self.x, self.o):
                if self.board[:,j].sum() == player*LENGTH:
                    self.winner = player
                    self.ended = True
                    return True
        # check diagonals
        for player in (self.x, self.o):
            # top-left -> bottom-right
            if self.board.trace() == player*LENGTH:
                self.winner = player
                self.ended = True
                return True
            # top-right -> bottom-left
            if np.flipr(self.board).trace() == player*LENGTH:
                self.winner = player
                self.ended = True
                return True
        # check if draw
        if np.all((self.board == 0) == False):
            # winner stays None
            self.winner = None
            self.ended = True
            return True
        # game is not over
        self.winner = None
        return False
    
    def draw_board(self):
        for i in xrange(LENGTH):
            print "------------"
            for j in xrange(LENGTH):
                print " ",
                if self.board[i,j] == self.x:
                    print "x",
                elif self.board[i,j] == self.o:
                    print "o",
                else:
                    print " ",
            print ""
        print "------------"


In [4]:
class Agent:
    def __init__(self, eps=0.1, alpha=0.5):
        self.eps = eps # probability of choosing random action instead of greedy
        self.alpha = alpha
        self.verbose = False
        self.state_history = ()
        
    def setV(self, V):
        self.V = V
        
    def set_symbol(self, symbol):
        self.sym = sym
    
    def set_verbose(self, v):
        # if true will print value for each position on the board
        self.verbose = v
        
    def reset_history(self):
        self.state_history = []
        
    def take_action(self, env):
        # choose an action based on epsilon-greedy strategy
        r = np.random.rand()
        best_state = None
        if r < self.eps:
            # take a random action
            if self.verbose:
                print "Taking a random action"
            
            possible_moves = []
            for i in xrange(LENGTH):
                for j in xrange(LENGTH):
                    if env.is_empty(i, j):
                        possible_moves.append((i, j))
            idx = np.random.choice(len(possible_moves))
            next_move = possible_moves[idx]
        else:
            pos2value = {}
            next_move = None
            best_value = -1
            for i in xrange(LENGTH):
                for j in xrange(LENGTH):
                    if env.is_empty(i, j):
                        # what is the state if we made this move ?
                        env.board[i, j] = self.sym
                        state = env.get_state()
                        env.board[i, j] = 0 # don't forget to change it back !
                        pos2value[(i, j)] = self.V[state]
                        if self.V[state] > best_value:
                            best_value = self.V[state]
                            best_state = state
                            next_move = (i, j)
            # if verbose, draw the board w/ the values
            if self.verbose:
                print "Taking a greedy action"
                for i in xrange(LENGTH):
                    print "----------------"
                    for j in xrange(LENGTH):
                        if env.is_empty(i, j):
                            # print the value
                            print "%.2f|" % pos2value[(i, j)],
                        else:
                            print " ",
                            if env.board[i, j] == env.x:
                                print "x |",
                            elif env.board[i, j] == env.o:
                                print "o |",
                            else:
                                print " |",
                    print ""
                print "----------------"
    
    def update_state_history(self, s):
        # cannot put this in take_action, because take_action only happens once
        # every other iteration for each player
        # state history needs to be updated every iteration
        # s = env.get_state() # don't want to do this twice so pass it in
        self.state_history.append(s)
        
    def update(self, env):
        # we want to BACKTRACK over the states, so that:
        # V(prev_state) = V(prev_state) + alpha*(V(next_state) - V(prev_state))
        # where V(next_state) = reward if it's the most current_state
        #
        # NOTE: we only do this at the end of an episode
        # not so for all the algorithms we will study
        reward = env.reward(self.sym)
        target = reward
        for prev in reversed(self.state_history):
            value = self.V[prev] + self.alpha * (target - self.V[prev])
            self.V[prev] = value
            target = value
        self.reset_history()
        

In [5]:
class Human:
    def __init__(self):
        pass
    
    def set_symbol(self, sym):
        self.sym = sym
    
    def take_action(self, env):
        while True:
            # break if we make a legal mov
            move = raw_input("Enter coordinates i,j for your next move (i, j=0..2): ")
            i, j = move.split(",")
            i = int(i)
            j = int(j)
            if env.is_empty(i, j):
                env.board[i, j] = self.sym
                break
    
    def update(self, env):
        pass
    
    def update_state_history(self, s):
        pass


# Training the AI

In [8]:
# players (agents)
p1 = Agent()
p2 = Agent()

# set initial V for p1 and p2
env = Environment()
state_winner_triples = get_state_hash_and_winner(env)

Vx = initialV_x(env, state_winner_triples)
p1.setV(Vx)
Vo = initialV_o(env, state_winner_triples)
p2.setV(Vo)

# give each player their symbol
p1.set_symbol(env.x)
p2.set_symbol(env.o)


# playin !
T = 10
for t in xrange(T):
    if t % 10 == 0:
        print t
    play_game(p1, p2, Environment())


NameError: name 'get_state_hash_and_winner' is not defined

# Playing against the AI

In [9]:
human = Human()
human.set_symbol(env.o)

while True:
    p1.set_verbose(True)
    play_game(p1, human, Environment(), draw=2)
    # making the AI player 1 allow to see which starting moves it takes (center ?)
    answer = raw_input("Play again ? [Y/n]: ")
    if answer and answer.lower()[0] = 'n':
        break

UnboundLocalError: local variable 'player' referenced before assignment